<a href="https://colab.research.google.com/github/valentingorce/tp_centrale/blob/main/Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Information Retrieval
## Introduction to search engines

### DAY 2: Teacher version
### Implementing a search engine

The goal of this second session is to implement a first architecture of a search engine on the previously introduced dataset (stackexchange-datascience). If you missed the first session or if you did not saved the dataset, please reload the first session's notebook to download it. 

If you need some ifnormation about the dataset, it should be available here : https://archive.org/details/stackexchange

The notebook is divided into several steps:
-	Implement the indexation
-	Implement the search method
-	Define a ranking strategy and implement it
-	Suggest some improvements of the search engine



## Initialisation

In [1]:
!pip install ttable

  Using cached ttable-0.6.4.tar.gz (122 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for ttable
Failed to build ttable


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [146 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\tt
      copying tt\version.py -> build\lib.win-amd64-cpython-310\tt
      copying tt\__init__.py -> build\lib.win-amd64-cpython-310\tt
      copying tt\__main__.py -> build\lib.win-amd64-cpython-310\tt
      creating build\lib.win-amd64-cpython-310\tt\cli
      copying tt\cli\core.py -> build\lib.win-amd64-cpython-310\tt\cli
      copying tt\cli\utils.py -> build\lib.win-amd64-cpython-310\tt\cli
      copying tt\cli\__init__.py -> build\lib.win-amd64-cpython-310\tt\cli
      creating build\lib.win-amd64-cpython-310\tt\definitions
      copying tt\definitions\grammar.py -> build\lib.win-amd64-cpython-310\tt\definitions
      copying tt\definitions\operands.py -> b

In [4]:
import pandas as pd
import re
import os
import math
import numpy as np
from sklearn.metrics import mean_squared_error
from tt import BooleanExpression
from itertools import product
import numpy as np

In [ ]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

In [4]:
DATA_PATH = '/content/drive/MyDrive/TP Centrale/data' 


**Important :**

An Excel file for testing the evaluation part is available in the gitlab repo : evaluation_search_engine_post_queries_ranking_EI_CS.xlsx

If you work on Colab, we advice you to push it directly on your Google Drive directory.

# Implement the indexation
As you might already know, for a search engine to work properly an index of the documents must be created. Here we will keep it in python, and try to use only common libraries to keep it simple.

Once created, the index will be used to match the query with the documents. As a result, there are several ways to build an index, using statistical, boolean, semantic indexation...

First of, let's make a naive one that will consist in breaking down each document into a set of the words it contains.

In [5]:
def extract_words(text:str)->list:
  return list(set(text.split(sep=" ")))

In [6]:
# test
s = "The cat is sat on the mat. The dog is laid on the mat."
assert extract_words(s).sort()==["The","cat","is","sat","on","the","mat.","dog","laid"].sort()

As you may notice, there are several problems with the previous implementation. First, "The" and "the" aren't considered the same, the "." is kept at the the end of "mat." as any other punctuation character... 

Re-implement this function with some basic preprocessing to avoid these issues.

In [7]:
def extract_words(text:str)->list:
  res = text.lower().split(sep=" ")
  for i in range(len(res)):
    res[i] = res[i].rstrip(".!?,;:")
  return list(set(res))

In [8]:
# test
assert extract_words(s).sort()==["the","cat","is","sat","on","mat","dog","laid"].sort()

Now you sould be able to create your index table. For now we will just make a dataframe with two columns: [raw_text, words].

In [9]:
import pandas as pd

def index_docs(docs:list[str])->pd.DataFrame:
  data = [[txt, extract_words(txt)] for txt in docs]
  return pd.DataFrame(data)

In [ ]:
# test
L = [s, "Hello World!", "Goodbye", "How are you?"]
index_docs(L)

Now, let's try it on the dataset:

In [ ]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

For our first version of the indexation mechanism, we will simply use the "body" of the posts. To have a better search engine, the title and other metadata aswell could be used aswell. Finally, not all the XML files have a "body" feature, so for the search engine to retrieve information from any of the files you will need to implement another way to index.

But first, let's start with "body". There is more to preprocess than before, indeed, there are html tags such as "<p>" for instance. They are not useful for us, because users won't use them in their queries. So we first need to remove them.

In [1]:
import re

def remove_tags(text: str) -> str:
    cleaned_text = re.sub(r'<.*?>', ' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remove extra whitespaces
    cleaned_text = cleaned_text.strip()  # Remove leading/trailing whitespaces
    return cleaned_text



In [2]:
# test
text = "<h1>Title</h1><p>This is a <b>sample</b> text.</p>"
cleaned_text = remove_tags(text)
print(cleaned_text)

remove_tags('<p>Hello World!\nI am making a search engine.<p>')

Title This is a sample text.


'Hello World! I am making a search engine.'

In [ ]:
clean_posts = posts[['Id','Body']]
clean_posts['Clean Body'] = clean_posts['Body'].fillna('').apply(remove_tags)
clean_posts

In [ ]:
clean_posts['words'] = clean_posts['Clean Body'].apply(extract_words)
clean_posts

## Zipf Law

A way of analyzing a corpus is to draw the zipf law

In [17]:
# TODO : Draw Zipf Law on the Posts Corpus

## Inverted Index

Now, we want to go further on the indexing and build an inverted index. Inverted index is a dictionary where the keys are the words of the vocabulary and the values are the documents containing these words. Reducing the size of the vocabulary is a relevant first step when building an inverted index. Here, we will focus on the creation of the index, we leave you the optimisation steps :)

In [18]:
def create_index(posts:pd.DataFrame)-> set:
  # TODO
  
  return 

In [19]:
inverted_index = create_index(clean_posts.iloc[0:5000])

#### Well Done, you've indexed the dataset! 
Don't hesitate to save your indexes in txt or pickle file

---
# Implement the search method

A naive method would be to count the number of words in common between the query and each posts. Then to rank the posts you could directly select the post who maximize the number of common words. Let's implement this approach :

In [21]:
# Implement the word_in_index function 
# Inputs : a word (str) & a list of words
# Output : pandas series of 1 if the word is in the list, else 0

def word_in_index(word, word_list_index):
  # TODO

  return

In [22]:
# Implement the function which run through a pandas series and count the number of word in common
# Use extract_words method, apply method with word_in_index function
# Inputs : the query (str) & pandas series of strings
# Output : Pandas series counting the number of common words between the query and each string in word_serie

def count_common_words(query, word_serie):
  # TODO
  
  return


In [23]:

def rank_top_query(query, df, top=5):
  # TODO


  return 

In [ ]:
rank_top_query(query="testing the query in python", df=clean_posts, top=5)

Testez plusieurs requêtes et critiquez les résultats obtenus.

Quels sont les pros and cons de cette méthodes. Vous l'indiquerez sur le rapport avec vos réflexions pour l'améliorer.

Next, you have to implement the first improvements you find in the search method to get most relevant results 

In [25]:

def remove_stop_words(l_txt: list) -> list:
  # TODO

  return 

## Boolean Search

Thanks to the ttable library, implement a boolean search method

In [ ]:
def boolean_search(query):
  # TODO

  return

## Probabilistic search

Implement the MIB or BM25 method of searching

In [ ]:
def probabilistic_search(query):
  # TODO

  return

Compare the naive method with your improvements and the boolean and probabilistic search. (report)



---



---




# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.



We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the xlsx file : . The final score will be the mean quadratic error of the queries.


Explication for the xlsx file :

We propose you a Excel file with some posts and a mesure of relevancy for the queries

- First column is the post Id,
- Columns starting by query are the queries you have to test.
- The values in this columns are the rank of relevancy of the post in regard with the query.
- The missing values indicates you should not take into account the post


You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [ ]:
# Read Relevancy CSV
df_relevancy = pd.read_excel("/content/drive/MyDrive/TP Centrale/evaluation_search_engine_post_queries_ranking_EI_CS.xlsx")

In [ ]:
def calculate_ndgc(query_col="query", output_col="query_output"):
  # TODO

  return

